In [21]:
import sys
# sys.path.append(r'E:/0code')
sys.path.append(r'/home/wyf/0code')
# sys.path.append(r'/home/wangyf226/0code')
# sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/pyml')

In [22]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
import gensim
import re
import smart_open
from pyml.feature_extraction.text import CountVectorizer
from pyml.linear_model.classification import LogisticClassifier
from pyml.neighbors.classification import KNeighborsClassifier
from pyml.metrics.classification import precision_score
from pyml.model_selection import KFold
from pyml.preprocessing import StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 读取数据集
1. 读取训练数据和测试数据为字符串的列表
2. 读取训练集label，并转换为数字格式

In [4]:
def read_train_text_to_list(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        contents = f.readlines()
    lines = [l.strip() for l in contents]
    return lines

In [5]:
# train_ori_X = read_train_text_to_list('../data/trainData.txt')
train_ori_Y = read_train_text_to_list('../data/trainLabel.txt')
train_ori_Y = np.array([int(y) for y in train_ori_Y])
# test_ori_X = read_train_text_to_list('../data/testData.txt')

In [6]:
def read_raw_documents(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="UTF-8") as f:
        for i, line in enumerate(f):
            if tokens_only:
                # 变小写，去标点符号，分词
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [7]:
train_sentences = list(read_raw_documents('../data/trainData.txt'))
test_sentences = list(read_raw_documents('../data/testData.txt', tokens_only=True))

# 数据预处理 & 特征工程
1. Count Vectors as feature
2. TF-IDF Vectors as festures
3. Word Embeddings as features
4. Text/NLP based features
5. Topic Models as features

In [8]:
vector_size = 50
model = gensim.models.doc2vec.Doc2Vec(vector_size=vector_size, min_count=2, epochs=40)

In [9]:
model.build_vocab(train_sentences)

In [10]:
model.train(train_sentences, total_examples=model.corpus_count, epochs=model.epochs)

In [11]:
n_train_samples = len(train_sentences)
n_test_samples = len(test_sentences)
vector_size = 50
train_X = np.zeros((n_train_samples, vector_size))
test_X = np.zeros((n_test_samples, vector_size))
for i in range(0, n_train_samples):
    train_X[i] = model.infer_vector(train_sentences[i][0])
for i in range(0, n_test_samples):
    test_X[i] = model.infer_vector(test_sentences[i])

In [12]:
train_Y = train_ori_Y

In [13]:
train_X.shape

(24000, 50)

# 交叉验证-寻找最好的参数

In [15]:
k_range = range(1,21)
k_splits = 10
ms = KFold(k_splits=k_splits)
k_scores = np.zeros((len(k_range)))
for train_indices, test_indices in ms.split(train_X):
    for k in k_range:
        clf = KNeighborsClassifier(k=k)
        clf.fit(train_X[train_indices], train_Y[train_indices])
        y_pred = clf.predict(train_X[test_indices])
        score = precision_score(train_Y[test_indices], y_pred)
        print('k : {} score: {}'.format(k, score))
        k_scores[k_range.index(k)] += score
avg_k_scores = k_scores/k_splits
print(avg_k_scores)
print("best k ", np.argmax(avg_k_scores)+1)

k : 1 score: 0.67875
k : 2 score: 0.67875
k : 3 score: 0.7083333333333334
k : 4 score: 0.7154166666666667
k : 5 score: 0.7270833333333333
k : 6 score: 0.7404166666666666
k : 7 score: 0.7408333333333333
k : 8 score: 0.7491666666666666
k : 9 score: 0.7475
k : 10 score: 0.7570833333333333
k : 11 score: 0.75125
k : 12 score: 0.7566666666666667
k : 13 score: 0.7529166666666667
k : 14 score: 0.75125
k : 15 score: 0.7570833333333333
k : 16 score: 0.7608333333333334
k : 17 score: 0.7570833333333333
k : 18 score: 0.765
k : 19 score: 0.75875
k : 20 score: 0.765
k : 1 score: 0.69125


KeyboardInterrupt: 

# 训练模型写入结果

In [19]:
k=15
clf = KNeighborsClassifier(k=k)
clf.fit(train_X,train_Y)

In [20]:
y_pred = clf.predict(test_X)
sub = pd.DataFrame(y_pred)
sub.to_csv('../results/'+'KNN-'+str(k)+'-'+ str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + ".csv", index=0, header=None, index_label=None)

KeyboardInterrupt: 

In [23]:
for k in [12,15,17,19,20]:
    clf = KNeighborsClassifier(k=k)
    clf.fit(train_X,train_Y)
    y_pred = clf.predict(test_X)
    sub = pd.DataFrame(y_pred)
    sub.to_csv('../results/'+'KNN-'+str(k)+'-'+ str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + ".csv", index=0, header=None, index_label=None)